### ⬛🟥Árvore RUBRO NEGRA⬛🟥

In [ ]:
# Constante das cores (divisão padrão da árvore)
RED = 1
BLACK = 0

#Temos que ter uma classe de node diferente, já que a organização do node da 2,3
#,4 não é a mesma da Rubro negra, pela presença de Cor, por exemplo.

class NodeRubroNegra:
    def __init__(self, data):
        self.data = data
        self.pai = None
        self.esquerda = None
        self.direita = None
        self.cor = RED  # Todo novo nó nasce vermelho por padrão!!!!

In [ ]:
class RedBlackTree:
    def __init__(self):
        # Sentinela (sentinela) representa folhas nulas e é sempre PRETO.
        self.sentinela = NodeRubroNegra(0)
        self.sentinela.cor = BLACK
        self.sentinela.esquerda = None
        self.sentinela.direita = None
        self.raiz = self.sentinela

    #OPERAÇÕES AUXILIARES

    def buscar(self, k):
        return self._buscar_helper(self.raiz, k)

    def _buscar_helper(self, node, key):
        if node == self.sentinela or key == node.data:
            return node
        if key < node.data:
            return self._buscar_helper(node.esquerda, key)
        return self._buscar_helper(node.direita, key)

    def _minimum(self, node):
        while node.esquerda != self.sentinela:
            node = node.esquerda
        return node

    # ROTAÇÕE s

    def rotacao_esquerda(self, x):
        y = x.direita
        x.direita = y.esquerda
        if y.esquerda != self.sentinela:
            y.esquerda.pai = x
        y.pai = x.pai
        if x.pai is None:
            self.raiz = y
        elif x == x.pai.esquerda:
            x.pai.esquerda = y
        else:
            x.pai.direita = y
        y.esquerda = x
        x.pai = y

    def rotacao_direita(self, x):
        y = x.esquerda
        x.esquerda = y.direita
        if y.direita != self.sentinela:
            y.direita.pai = x
        y.pai = x.pai
        if x.pai is None:
            self.raiz = y
        elif x == x.pai.direita:
            x.pai.direita = y
        else:
            x.pai.esquerda = y
        y.direita = x
        x.pai = y

    # inserção

    def insert(self, key):
        if self.buscar(key) != self.sentinela:
            print(f"Inserção ignorada: Chave {key} já existe.")
            return

        # Criação do nó
        node = NodeRubroNegra(key)
        node.pai = None
        node.data = key
        node.esquerda = self.sentinela
        node.direita = self.sentinela
        node.cor = RED

        #Inserção padrão árvore binária
        y = None
        x = self.raiz

        while x != self.sentinela:
            y = x
            if node.data < x.data:
                x = x.esquerda
            else:
                x = x.direita

        node.pai = y
        if y is None:
            self.raiz = node
        elif node.data < y.data:
            y.esquerda = node
        else:
            y.direita = node

        #Se for raiz, termina
        if node.pai is None:
            node.cor = BLACK
            return

        if node.pai.pai is None:
            return

        #Correção do balanceamento
        self._corrigir_insert(node)

    def _corrigir_insert(self, k):
        while k.pai.cor == RED:
            if k.pai == k.pai.pai.direita:
                u = k.pai.pai.esquerda # Tio
                if u.cor == RED:
                    # Caso 1: Tio Vermelho -> Recoração
                    u.cor = BLACK
                    k.pai.cor = BLACK
                    k.pai.pai.cor = RED
                    k = k.pai.pai
                else:
                    if k == k.pai.esquerda:
                        # Caso 2: Tio Preto e Triângulo -> Rotação no Pai
                        k = k.pai
                        self.rotacao_direita(k)
                    # Caso 3: Tio Preto e Linha -> Rotação no Avô + Cores
                    k.pai.cor = BLACK
                    k.pai.pai.cor = RED
                    self.rotacao_esquerda(k.pai.pai)
            else:
                u = k.pai.pai.direita # Tio
                if u.cor == RED:
                    u.cor = BLACK
                    k.pai.cor = BLACK
                    k.pai.pai.cor = RED
                    k = k.pai.pai
                else:
                    if k == k.pai.direita:
                        k = k.pai
                        self.rotacao_esquerda(k)
                    k.pai.cor = BLACK
                    k.pai.pai.cor = RED
                    self.rotacao_direita(k.pai.pai)
            if k == self.raiz:
                break
        self.raiz.cor = BLACK

    #REMOÇÃO

    def deletar_node(self, data):
        node = self.buscar(data)
        if node == self.sentinela:
            print(f"Chave {data} não encontrada para exclusão.")
            return
        self._delete_helper(node)

    def _trocarVertices(self, u, v):
        if u.pai is None:
            self.raiz = v
        elif u == u.pai.esquerda:
            u.pai.esquerda = v
        else:
            u.pai.direita = v
        v.pai = u.pai

    def _delete_helper(self, z):
        y = z
        y_original_cor = y.cor
        if z.esquerda == self.sentinela:
            x = z.direita
            self._trocarVertices(z, z.direita)
        elif z.direita == self.sentinela:
            x = z.esquerda
            self._trocarVertices(z, z.esquerda)
        else:
            y = self._minimum(z.direita)
            y_original_cor = y.cor
            x = y.direita
            if y.pai == z:
                x.pai = y
            else:
                self._trocarVertices(y, y.direita)
                y.direita = z.direita
                y.direita.pai = y

            self._trocarVertices(z, y)
            y.esquerda = z.esquerda
            y.esquerda.pai = y
            y.cor = z.cor

        if y_original_cor == BLACK:
            self._corrigir_delete(x)

    def _corrigir_delete(self, x):
        while x != self.raiz and x.cor == BLACK:
            if x == x.pai.esquerda:
                s = x.pai.direita
                if s.cor == RED:
                    s.cor = BLACK
                    x.pai.cor = RED
                    self.rotacao_esquerda(x.pai)
                    s = x.pai.direita

                if s.esquerda.cor == BLACK and s.direita.cor == BLACK:
                    s.cor = RED
                    x = x.pai
                else:
                    if s.direita.cor == BLACK:
                        s.esquerda.cor = BLACK
                        s.cor = RED
                        self.rotacao_direita(s)
                        s = x.pai.direita

                    s.cor = x.pai.cor
                    x.pai.cor = BLACK
                    s.direita.cor = BLACK
                    self.rotacao_esquerda(x.pai)
                    x = self.raiz
            else:
                s = x.pai.esquerda
                if s.cor == RED:
                    s.cor = BLACK
                    x.pai.cor = RED
                    self.rotacao_direita(x.pai)
                    s = x.pai.esquerda

                if s.direita.cor == BLACK and s.esquerda.cor == BLACK:
                    s.cor = RED
                    x = x.pai
                else:
                    if s.esquerda.cor == BLACK:
                        s.direita.cor = BLACK
                        s.cor = RED
                        self.rotacao_esquerda(s)
                        s = x.pai.esquerda

                    s.cor = x.pai.cor
                    x.pai.cor = BLACK
                    s.esquerda.cor = BLACK
                    self.rotacao_direita(x.pai)
                    x = self.raiz
        x.cor = BLACK

    # VISUALIZAÇÃO

    def printar_arvore(self):
        print("\n--- Visualização da Árvore ---")
        self.formatarPrint(self.raiz, "", True)
        print("------------------------------\n")

    def formatarPrint(self, node, indentacao, last):
        if node != self.sentinela:
            print(indentacao, end="")

            if last:
                print("R----", end="")
                indentacao += "   "
            else:
                print("L----", end="")
                indentacao += "|  "

            s_cor = "RED" if node.cor == RED else "BLACK"
            print(f"{node.data} ({s_cor})")
            self.formatarPrint(node.esquerda, indentacao, False)
            self.formatarPrint(node.direita, indentacao, True)

In [ ]:
arvore_flamengo = RedBlackTree()


dados = [
    50, 40, 30, 20, 10,       # Sequência descrescente (força rotações)
    60, 70, 80, 90, 100,      # Sequência crescente
    45, 42, 47,               # Casos de "joelho" (zig-zag)
    55, 65, 75, 85, 95, 5, 1, # Folhas nas pontas
    15, 25, 35                # Preenchimento interno
]

print(f"Inserindo {len(dados)} elementos na Árvore...")
for valor in dados:
    arvore_flamengo.insert(valor)

arvore_flamengo.printar_arvore()

In [ ]:
#testes p/ vizualizar

#Busca
busca = 42
print(f"Buscando o valor {busca}...")
no = arvore_flamengo.buscar(busca)
if no != arvore_flamengo.sentinela:
    cor = "Vermelho" if no.cor == RED else "Preto"
    print(f"   -> Encontrado! O nó {busca} é {cor}.")
else:
    print(f"   -> Valor {busca} não encontrado.")

print("-" * 30)

#repetidos
repetido = 50
print(f"Tentando inserir duplicado ({repetido})...")
arvore_flamengo.insert(repetido)

print("-" * 30)

# exclusão c/ rebalanceamento (nó preto c filhos)
alvo_remocao = 30

print(f"Removendo o valor {alvo_remocao}...")
arvore_flamengo.deletar_node(alvo_remocao)

print("\n--- Árvore Resultante Após Remoção ---")
arvore_flamengo.printar_arvore()